In [ ]:
%pip install -U \
  accelerate==0.26.1 \
  datasets==2.16.1 \
  peft==0.8.1 \
  transformers==4.37.2 \
  torch==2.1.0 \
  numpy==1.26.4

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.0.3 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
mizani 0.13.2 requires pandas>=2.2.0, but you have pandas 2.0.3 which is incompatible.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.
xarray 2025.1.2 requires pandas>=2.1, but you have pandas 2.0.3 which is incompatib

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch
import numpy

In [ ]:
base_model = "codellama/CodeLlama-7b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
!pip install -U numpy==1.25.2 pandas==2.0.3 datasets==2.16.1

  Using cached numpy-1.25.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.25.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.0.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.2 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.25.2 which is incompatible.
mizani 0.13.2 requires pandas>=2.2.0, but you have pandas 2.0.3 which is incompatible.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.2 which i

In [ ]:
from datasets import load_dataset

dataset = load_dataset('json', data_files= '/content/drive/MyDrive/Intelligent Coding Assistant_Masters Project/note_data.json', split='train')

In [ ]:
print(dataset)

In [ ]:
def tokenize_function(examples):
    inputs = []
    targets = []

    for main_topic, subtopics in zip(examples["MainTopic"], examples["SubTopics"]):
        # Build input prompt
        input_text = "### Instruction:\nGenerate a concise, structured note on the following topic.\n\n"
        input_text += f"# Topic: {main_topic}\n\n"
        for sub in subtopics:
            title = sub.get("Title", "")
            desc = sub.get("Description", "")
            code = sub.get("Code Snippets", "")
            if code:
                input_text += f"## {title}\n{desc}\n\n```python\n{code}\n```\n\n"
            else:
                input_text += f"## {title}\n{desc}\n\n"

        # Build target output — clean markdown-style note
        output_text = f"### Notes on {main_topic}:\n\n"
        for sub in subtopics:
            title = sub.get("Title", "")
            desc = sub.get("Description", "")
            output_text += f"**{title}**\n- {desc.strip()}\n\n"
            if "Code Snippets" in sub and sub["Code Snippets"]:
                output_text += f"```python\n{sub['Code Snippets'].strip()}\n```\n\n"

        inputs.append(input_text.strip())
        targets.append(output_text.strip())

    # Tokenize with labels
    tokenized = tokenizer(
        inputs,
        padding="max_length",
        truncation=True,
        max_length=512,
    )
    labels = tokenizer(
        targets,
        padding="max_length",
        truncation=True,
        max_length=512,
    )
    tokenized["labels"] = labels["input_ids"]
    return tokenized
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype= "auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
pip install peft

In [ ]:
from peft import prepare_model_for_kbit_training

# Prepare model for training when using 8-bit or 4-bit quantization
model = prepare_model_for_kbit_training(model)


In [ ]:
from peft import LoraConfig, get_peft_model

# Define LoRa configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"  # Ensure correct spelling
)
model.enable_input_require_grads()
# Apply LoRa to the model
model = get_peft_model(model, lora_config)

# ✅ Check if any parameters are trainable
model.print_trainable_parameters()


trainable params: 8,388,608 || all params: 6,746,935,296 || trainable%: 0.12433212461624295


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir= "./fine_tuned_codellama_notes",
    evaluation_strategy="no",  # ✅ Disable evaluation
    save_strategy="epoch",
    per_device_train_batch_size=1,  # ✅ Reduce batch size for memory efficiency
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    logging_dir="./logs",
    fp16=True,  # ✅ Enable mixed precision for better efficiency
    gradient_checkpointing=True,  # ✅ Saves VRAM
    save_total_limit=1,  # ✅ Prevents excessive checkpoint saving
    report_to="none"  # ✅ Ensures no W&B logging
)

In [ ]:
model.train()  # ✅ Put model in training mode


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32016, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
           

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # **Set to False for causal LM**
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,  # ✅ Ensure correct data collator is used
)


trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacty of 22.16 GiB of which 51.38 MiB is free. Process 65071 has 22.11 GiB memory in use. Of the allocated memory 21.91 GiB is allocated by PyTorch, and 18.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model.save_pretrained("./fine_tuned_codellama_notes")
tokenizer.save_pretrained("./fine_tuned_codellama_notes")


In [ ]:
#Download trained model for notes
"""import shutil

shutil.make_archive("fine_tuned_codellama_notes", 'zip', "./fine_tuned_codellama_notes")

from google.colab import files
files.download("fine_tuned_codellama_notes.zip")"""


FileNotFoundError: [Errno 2] No such file or directory: './fine_tuned_codellama_notes'

In [ ]:
 pip install transformers

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

#base_
model = AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-7b-Instruct-hf",
    device_map="auto",
    torch_dtype=torch.float16
)

#model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/fine_tuned_codellama_notes")
#model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")

note_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Zero-shot prompt
"""prompt = """
"""You are a Programming teacher. Generate detailed class notes.

Topic: Array
Notes:"""


"""

output = note_generator(prompt, max_new_tokens=300, do_sample=True, temperature=0.7)
print(output[0]["generated_text"])"""


'\n\noutput = note_generator(prompt, max_new_tokens=300, do_sample=True, temperature=0.7)\nprint(output[0]["generated_text"])'

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
def make_note_prompt(dataset, example_indices_full, example_index_to_generate, max_subtopics=5):
    prompt = ""

    for idx in example_indices_full:
        topic = dataset[idx]['MainTopic']
        subtopics = dataset[idx]['SubTopics'][:max_subtopics]

        subtopics_text = ""
        for sub in subtopics:
            title = sub.get("Title", "")
            desc = sub.get("Description", "")
            code = sub.get("Code Snippets", "")
            subtopics_text += f"## {title}\n{desc}\n"
            if code:
                subtopics_text += f"\n```python\n{code}\n```\n"

        prompt += f"""You are a Programming teacher. Generate detailed class notes.

Topic: {topic}

Content:
{subtopics_text}

Notes:


"""

    # generation part
    topic = dataset[example_index_to_generate]['MainTopic']
    subtopics = dataset[example_index_to_generate]['SubTopics'][:max_subtopics]

    subtopics_text = ""
    for sub in subtopics:
        title = sub.get("Title", "")
        desc = sub.get("Description", "")
        code = sub.get("Code Snippets", "")
        subtopics_text += f"## {title}\n{desc}\n"
        if code:
            subtopics_text += f"\n```python\n{code}\n```\n"

    prompt += f"""You are a Programming teacher. Generate detailed class notes.

Topic: {topic}

Content:
{subtopics_text}

Notes:"""

    return prompt




In [ ]:
prompt = make_note_prompt(dataset, [0], 1, max_subtopics=5)
output = note_generator(prompt, max_new_tokens=200, do_sample=True, temperature=0.7)
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a Programming teacher. Generate detailed class notes.

Topic: Array

Content:
## Basic terminologies of Array
Array Index:  In an array, elements are identified by their indexes. Array index starts from 0. Array element:  Elements are items stored in an array and can be accessed by their index. Array Length:  The length of an array is determined by the number of elements it can contain.
## Memory representation of Array
In an array, all the elements are stored in contiguous memory locations. So, if we initialize an array, the elements will be allocated sequentially in memory. This allows for efficient access and manipulation of elements. 
## Declaration of Array
Arrays can be declared in various ways in different languages. For better illustration, below are some language-specific array declarations:

```python
# In Python, all types of lists are created same way
arr = []
```
## Initialization of Array
Arrays can be initialized in different ways in different languages. Below ar

In [ ]:
def make_one_shot_then_custom_topic_prompt(dataset, example_indices_full, custom_topic, max_subtopics=5):
    prompt = ""

    # One-shot example from dataset
    example = dataset[example_indices_full]
    topic = example['MainTopic']
    subtopics = example['SubTopics']

    example_content = ""
    for sub in subtopics:
        title = sub.get("Title", "")
        desc = sub.get("Description", "")
        code = sub.get("Code Snippets", "")
        example_content += f"## {title}\n{desc}\n"
        if code:
            example_content += f"\n```python\n{code}\n```\n"

    prompt += f"""You are a Programming teacher. Generate detailed class notes including subtopics, explanations, and code examples only in Python3 code.

Topic: {topic}

Content:
{example_content}

Notes:


"""

    # Custom generation part — only topic provided
    prompt += f"""You are a Programming teacher. Generate detailed class notes including subtopics, explanations, and code examples only in Python3 code.

Topic: {custom_topic}

Content:
"""

    return prompt



In [ ]:
custom_topic = "Recursion"
prompt = make_one_shot_then_custom_topic_prompt(dataset, example_indices_full=0, custom_topic=custom_topic, max_subtopics=5)

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=1000,
    do_sample=True,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
#add max_new-token

You are a Programming teacher. Generate detailed class notes including subtopics, explanations, and code examples only in Python3 code.

Topic: Array

Content:
## Basic terminologies of Array
Array Index:  In an array, elements are identified by their indexes. Array index starts from 0. Array element:  Elements are items stored in an array and can be accessed by their index. Array Length:  The length of an array is determined by the number of elements it can contain.
## Memory representation of Array
In an array, all the elements are stored in contiguous memory locations. So, if we initialize an array, the elements will be allocated sequentially in memory. This allows for efficient access and manipulation of elements. 
## Declaration of Array
Arrays can be declared in various ways in different languages. For better illustration, below are some language-specific array declarations:

```python
# In Python, all types of lists are created same way
arr = []
```
## Initialization of Array
Ar